In [ ]:
!pip install --upgrade gensim

In [ ]:
# imports
import pandas as pd
import numpy as np
from google.cloud import storage
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
import gensim.downloader as api

%matplotlib inline

In [ ]:
# Get files from GCS bucket
BUCKET_NAME = 'salary-data'
client = storage.Client()
bucket = client.get_bucket(BUCKET_NAME)

blobs = bucket.list_blobs()
files = []
for blob in blobs:
    files.append(blob.name)
    
print(files)

In [ ]:
# Read data
df_TTWA = pd.read_csv('gs://{}/{}'.format(BUCKET_NAME,files[0]))
df_train = pd.read_csv('gs://{}/{}'.format(BUCKET_NAME,files[3]))

In [8]:
from functions_6 import TTWA_county_feature
df_loc = TTWA_county_feature(df_train,df_TTWA,True) 


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/jupyter/salary-notebooks/functions_6.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Loc['TTWA_County'] = df_Loc.TTWA
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [9]:
TTWA_county = df_loc.TTWA_County
df_all = pd.concat([df_train, TTWA_county], axis = 1)
df_all

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,TTWA,County,TTWA_County
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,NaN,Surrey,Surrey
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk,Glasgow,NaN,Glasgow
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk,NaN,Hampshire,Hampshire
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk,NaN,Surrey,Surrey
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,NaN,Surrey,Surrey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244763,72705211,TEACHER OF SCIENCE,Position: Qualified Teacher Subject/Specialism...,Swindon,Swindon,NaN,contract,NaN,Teaching Jobs,450 - 500 per week,22800,hays.co.uk,Swindon,NaN,Swindon
244764,72705212,TEACHER OF BUSINESS STUDIES AND ICT,Position: Qualified Teacher or NQT Subject/Spe...,Swindon,Swindon,NaN,contract,NaN,Teaching Jobs,450 - 500 per week,22800,hays.co.uk,Swindon,NaN,Swindon
244765,72705213,ENGLISH TEACHER,Position: Qualified Teacher Subject/Specialism...,Swindon,Swindon,NaN,contract,NaN,Teaching Jobs,450 - 500 per week,22800,hays.co.uk,Swindon,NaN,Swindon
244766,72705216,SUPPLY TEACHERS,Position: Qualified Teacher Subject/Specialism...,Wiltshire,Wiltshire,NaN,contract,NaN,Teaching Jobs,450 to 500 per week,22800,hays.co.uk,NaN,Wiltshire,Wiltshire


In [ ]:
client = storage.Client()
bucket = client.get_bucket(BUCKET_NAME)
bucket.blob('data_location_buckets.csv').upload_from_string(df_all.to_csv(), 'text/csv')

## Deal with NaN values

In [ ]:
# Fill in missing values for Contract and Company
df_all.ContractType = df_train.ContractType.fillna('Missing')
df_all.ContractTime = df_train.ContractTime.fillna('Missing')
df_all.Company = df_train.Company.fillna('Missing')
# Drop other NaN values
df_all = df_all.dropna()
df_all.isnull().sum()

## NLP of Description

In [72]:
import functions_12 as fn
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

In [59]:
def first_NLP(df):
    word_list = []
    for index, row in df.iterrows():
        bag = row.FullDescription
        ns = fn.remove_stop_words(bag, unique = False)
        lemma = fn.lemmatize_words(ns)
        word_list.append(lemma)
    return word_list

In [54]:
# Create a train, val, test split

X_train_full, X_test, y_train_full, y_test = train_test_split(
                                df_train.drop(['SalaryNormalized','SalaryRaw'], axis =1),
                                df_train.SalaryNormalized, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
                                                X_train_full,
                                                y_train_full, test_size=0.2, random_state=42)

In [65]:
# Remove grammer, capitals, and stop words
X_train_words = first_NLP(X_train)
X_val_words = first_NLP(X_val)
X_test_words = first_NLP(X_test)

In [67]:
# create dictionary of only the TRAIN text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_words)
word_dictionary = tokenizer.word_index

# Convert text to indexed list
X_train_seq = tokenizer.texts_to_sequences(X_train_words)
X_val_seq = tokenizer.texts_to_sequences(X_val_words)
X_test_seq = tokenizer.texts_to_sequences(X_test_words)

In [74]:
# Pad sequences

max_length = fn.get_max_length(X_train_seq, X_test_seq, X_val_seq)

X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq,maxlen=max_length, padding='post')
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq,maxlen=max_length, padding='post')
X_val_padded = tf.keras.preprocessing.sequence.pad_sequences(X_val_seq,maxlen=max_length, padding='post')

In [78]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

149126


## Set up model

In [84]:
# Load word embeddings from GloVe

glove_model = api.load('glove-twitter-25')
sample_glove_embedding=glove_model['computer']

In [89]:
type(glove_model)

gensim.models.keyedvectors.Word2VecKeyedVectors